## Prep Postgres and Ducklake

In [3]:
%pip install duckdb --quiet
%pip install --upgrade s3fs --quiet
%pip install connectorx adbc_driver_postgresql psycopg2-binary --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

import duckdb
import polars as pl
from dotenv import load_dotenv

load_dotenv()
pass

In [5]:
con = duckdb.connect()
con.execute("install ducklake;")
con.execute("install postgres;")

con.execute(
    """
        CREATE SECRET (
            TYPE postgres,
            HOST '{host}',
            PORT {port},
            DATABASE {db},
            USER '{user}',
            PASSWORD '{password}'
        );
    """.format(
        host=os.getenv("POSTGRES_HOST"),
        port=os.getenv("POSTGRES_PORT"),
        user=os.getenv("DUCKLAKE_PG_LOGIN"),
        password=os.getenv("DUCKLAKE_PG_PASS"),
        db="ducklake_catalog",
    )
)

con.execute(
    """
        CREATE OR REPLACE SECRET secret (
            TYPE s3,
            ENDPOINT '{endpoint}',
            KEY_ID '{key_id}',
            SECRET '{secret}',
            URL_STYLE 'path',
            USE_SSL 'false'
        );
    """.format(
        endpoint="seaweedfs-s3.seaweedfs:8333",
        key_id=os.getenv("S3_AWS_ACCESS_KEY_ID"),
        secret=os.getenv("S3_AWS_SECRET_ACCESS_KEY"),
    )
)

con.execute(
    """
        ATTACH 'ducklake:postgres:dbname=ducklake_catalog host={host}' AS my_ducklake
            (DATA_PATH 's3://ducklake/');
        USE my_ducklake;
    """.format(host=os.getenv("POSTGRES_HOST"))
)

## Run some queries agains hard drive data in DuckLake and PostgreSQL

### Count

In [14]:
%%time
## estimate not precise but fast
df = pl.read_database_uri(
    query="SELECT reltuples::bigint FROM pg_class WHERE oid = 'raw.hard_drive_data'::regclass;",
    uri=os.getenv("POSTGRES_CONN_STR"),
)
df

CPU times: user 2.3 ms, sys: 1.18 ms, total: 3.48 ms
Wall time: 9.6 ms


reltuples
i64
56592856


In [15]:
%%time
con.execute(
    """
        SELECT count(*)
        FROM hard_drive_data;
    """
).fetchall()

CPU times: user 138 ms, sys: 19.4 ms, total: 157 ms
Wall time: 159 ms


[(56608028,)]

### Top 3 models by total capacity on 2025-01-01

Postgres has index on date

In [10]:
%time
df = pl.read_database_uri(
    query="""
        SELECT model, sum(capacity_bytes) / 1024^5 as total_capacity_pib_per_model
        FROM raw.hard_drive_data
        WHERE date = '2025-01-01'
        GROUP BY model
        ORDER BY total_capacity_pib_per_model DESC
        LIMIT 3
        """,
    uri=os.getenv("POSTGRES_CONN_STR"),
)
df

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


model,total_capacity_pib_per_model
str,f64
"""WDC WUH722222ALE6L4""",586.243143
"""TOSHIBA MG08ACA16TA""",570.29949
"""ST16000NM001G""",477.511597


In [11]:
%time
con.execute(
    """
        SELECT model, sum(capacity_bytes) / 1024^5 as total_capacity_pib_per_model,
        FROM hard_drive_data
        WHERE date = '2025-01-01'
        GROUP BY model
        ORDER BY 2 DESC
        LIMIT 3;
    """
).fetch_df()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


,model,total_capacity_pib_per_model
0,WDC WUH722222ALE6L4,586.243143
1,TOSHIBA MG08ACA16TA,570.299490
2,ST16000NM001G,477.511597
